In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('olist').getOrCreate()

hdfs_path = '/olist_ka_data/'


olist_customers_df = spark.read.csv("/olist_ka_data/olist_customers_dataset.csv", header=True, inferSchema=True)
olist_geolocation_df = spark.read.csv("/olist_ka_data/olist_geolocation_dataset.csv", header=True, inferSchema=True)
olist_order_items_df = spark.read.csv("/olist_ka_data/olist_order_items_dataset.csv", header=True, inferSchema=True)
olist_order_payments_df = spark.read.csv("/olist_ka_data/olist_order_payments_dataset.csv", header=True, inferSchema=True)
olist_order_reviews_df = spark.read.csv("/olist_ka_data/olist_order_reviews_dataset.csv", header=True, inferSchema=True)
olist_orders_df = spark.read.csv("/olist_ka_data/olist_orders_dataset.csv", header=True, inferSchema=True)
olist_products_df = spark.read.csv("/olist_ka_data/olist_products_dataset.csv", header=True, inferSchema=True)
olist_sellers_df = spark.read.csv("/olist_ka_data/olist_sellers_dataset.csv", header=True, inferSchema=True)
product_category_name_translation_df = spark.read.csv("/olist_ka_data/product_category_name_translation.csv", header=True, inferSchema=True)


25/03/09 03:02:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
from pyspark.sql.functions import *

In [3]:
olist_orders_df.cache()

DataFrame[order_id: string, customer_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp]

In [4]:
order_items_joined_df = olist_orders_df.join(olist_order_items_df,'order_id','inner')

In [5]:
order_items_product_df = order_items_joined_df.join(olist_products_df,'product_id','inner')

In [6]:
order_items_product_seller_df = order_items_product_df.join(olist_sellers_df,'seller_id','inner')

In [7]:
full_orders_df = order_items_product_seller_df.join(olist_customers_df,'customer_id','inner')

In [8]:
full_orders_df = full_orders_df.join(olist_geolocation_df,full_orders_df.customer_zip_code_prefix ==olist_geolocation_df.geolocation_zip_code_prefix ,'inner')

In [9]:
full_orders_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [10]:
full_orders_df= full_orders_df.join(broadcast(olist_order_reviews_df),'order_id','left')

In [11]:
full_orders_df = full_orders_df.join(olist_order_payments_df,'order_id','left')

In [12]:
full_orders_df.cache()

25/03/09 03:05:34 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[order_id: string, customer_id: string, seller_id: string, product_id: string, order_status: string, order_purchase_timestamp: timestamp, order_approved_at: timestamp, order_delivered_carrier_date: timestamp, order_delivered_customer_date: timestamp, order_estimated_delivery_date: timestamp, order_item_id: int, shipping_limit_date: timestamp, price: double, freight_value: double, product_category_name: string, product_name_lenght: int, product_description_lenght: int, product_photos_qty: int, product_weight_g: int, product_length_cm: int, product_height_cm: int, product_width_cm: int, seller_zip_code_prefix: int, seller_city: string, seller_state: string, customer_unique_id: string, customer_zip_code_prefix: int, customer_city: string, customer_state: string, geolocation_zip_code_prefix: int, geolocation_lat: double, geolocation_lng: double, geolocation_city: string, geolocation_state: string, review_id: string, review_score: string, review_comment_title: string, review_commen

In [13]:
# Total Revnue Per seller

seller_revenue_df = full_orders_df.groupBy('seller_id').agg(sum('price'))

In [14]:
seller_revenue_df.show()

+--------------------+--------------------+
|           seller_id|          sum(price)|
+--------------------+--------------------+
|e63e8bfa530fb1691...|  219481.00000000035|
|ff063b022a9a0aab9...|           1860394.0|
|a49928bcdf77c55c6...|  1220624.6000000052|
|33ac3e28642ab8bda...|   615628.8499999995|
|7aa4334be125fcdd2...|   2509245.989999996|
|537eb890efff034a8...|   894311.9000000025|
|b39d7fe263ef46960...|   729876.4000000011|
|ef0ace09169ac0905...|  2261367.7000000025|
|f457c46070d02cadd...|   3812449.499999996|
|87b740daf17b5d1be...|   2129079.519999993|
|9c0e69c7bf2619675...|   5790971.859999996|
|a420f60ff1aa9acc8...|   999172.5000000001|
|fe2032dab1a61af87...|1.0376350110000022E7|
|fad44952713764836...|            934189.0|
|7a67c85e85bb2ce85...|2.0312442919999894E7|
|972d0f9cf61b499a4...|  1255780.7600000019|
|c24173065483e421f...|   29390.27999999993|
|ca7c6bd577e559472...|             21885.5|
|6426d21aca402a131...|  210373.14000000054|
|9d681c7e12db302cb...|   75827.6

In [17]:
seller_revenue_df.show()

+--------------------+--------------------+
|           seller_id|          sum(price)|
+--------------------+--------------------+
|0ea22c1cfbdc755f8...|  1747285.3000000052|
|e63e8bfa530fb1691...|  219481.00000000032|
|33ac3e28642ab8bda...|   615628.8499999995|
|7aa4334be125fcdd2...|   2509245.989999998|
|f457c46070d02cadd...|  3812449.4999999963|
|5a6a02f419701fc8c...|   69180.89999999992|
|7a67c85e85bb2ce85...|2.0312442919999905E7|
|5dceca129747e92ff...|1.4909400350000003E7|
|729f06993dac8e860...|  2774183.5599999926|
|d2374cbcbb3ca4ab1...|   3375370.150000015|
|1e8b33f18b4f7598d...|  3218885.1999999965|
|70a12e78e608ac311...|   4838320.179999991|
|238fac594e170b59c...|            480044.0|
|76d5af76d0271110f...|          4672492.77|
|e26901d5ab434ce92...|  2040715.4000000204|
|0c7533c71df861ec5...|  463382.39999999857|
|8e8a7ce9f2f970dc0...|           1176715.0|
|cac4c8e7b1ca6252d...|   661026.2600000006|
|0ebd97a106433a45a...|  396542.80000000127|
|bd43e172d599bed47...|  3635.840

In [15]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

# HW

In [17]:
# Total Orders Per Customer

customer_order_df = full_orders_df.groupBy('customer_id')\
.agg(
    count('order_id').alias('total_orders')
).orderBy('total_orders',ascending=False)

customer_order_df.show()

+--------------------+------------+
|         customer_id|total_orders|
+--------------------+------------+
|351e40989da90e704...|       11427|
|50920f8cd0681fd86...|       10752|
|9b43e2a62de9bab3a...|        8556|
|270c23a11d024a44c...|        8001|
|5c87184371002d49e...|        6876|
|d3e82ccec3cb5f956...|        6876|
|d5f2b3f597c7ccafb...|        6706|
|c2f18647725395af4...|        6612|
|24e7dc2ff8c071263...|        6597|
|7bb57d182bdc11653...|        6258|
|d22f25a9fadfb1abb...|        6072|
|63b964e79dee32a35...|        6072|
|1ff773612ab8934db...|        5820|
|13aa59158da63ba0e...|        5206|
|78fc46047c4a639e8...|        5200|
|dd3f1762eb601f41c...|        4992|
|a193aa8d905b8e246...|        4896|
|9eb3d566e87289dcb...|        4872|
|2ba91e12e5e4c9f56...|        4752|
|1b2ab6eda1946a6ff...|        4728|
+--------------------+------------+
only showing top 20 rows



In [ ]:

# High Value Customers

# DIY

In [19]:
# Most Sold Products

top_products = full_orders_df.groupBy('product_id')\
.agg(
    count('order_id').alias('total_sold')
).orderBy(desc('total_sold'))\
          .limit(10)

In [20]:
top_products.show()

+--------------------+----------+
|          product_id|total_sold|
+--------------------+----------+
|aca2eb7d00ea1a7b8...|     86738|
|422879e10f4668299...|     81109|
|99a4788cb24856965...|     78774|
|389d119b48cf3043d...|     60247|
|d1c427060a0f73f6b...|     59273|
|368c6c730842d7801...|     58355|
|53759a2ecddad2bb8...|     52654|
|53b36df67ebb7c415...|     52104|
|154e7e31ebfa09220...|     42697|
|3dd2a17168ec895c7...|     40787|
+--------------------+----------+



In [22]:
# Product with most reviews
full_orders_df[['product_id','review_score']].show()

+--------------------+------------+
|          product_id|review_score|
+--------------------+------------+
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
|b12f82804a690dfab...|           5|
+--------------------+------------+
only showing top 20 rows



In [ ]:
# Average Review Score by Seller

In [ ]:
# Window Functions in SQL
# Rank & DenseRank

# Advance Aggregation

In [104]:
customers_spending_df

NameError: name 'customers_spending_df' is not defined

In [105]:
# Total Revenue and AOV(Average order Value) per Customer

customers_spending_df = full_orders_df.groupBy('customer_id')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_revenue'),
    round(avg('price'),2).alias('AOV')).orderBy('total_revenue',ascending=False)

In [22]:
6662844.0/6874

969.2819319173698

In [106]:
customers_spending_df.show()

+--------------------+------------+------------------+-------+
|         customer_id|total_orders|     total_revenue|    AOV|
+--------------------+------------+------------------+-------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|
|695476b5848d64ba0...|         687|1820543.1299999943|2649.99|
|73236a0796f53d60d...|         832|         1755520.0| 2110.0|
|cc803a2c412833101...|         762|         1676400.0| 2200.0|
|1ff773612ab8934db...|        5820|1658641.7999999512| 

In [24]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [27]:
# Seller Performance Metrics (Revenue, Average Review, count)

seller_performace_df = full_orders_df.groupBy('seller_id')\
.agg(
    count('order_id').alias('total_orders'),
    sum('price').alias('total_revenue'),
    round(avg('review_score'),2).alias('avg review')
).orderBy('total_revenue',ascending=False)


In [28]:
seller_performace_df.show()

+--------------------+------------+--------------------+----------+
|           seller_id|total_orders|       total_revenue|avg review|
+--------------------+------------+--------------------+----------+
|4869f7a5dfa277a7d...|      184586|       3.613854732E7|      4.09|
|53243585a1d6dc264...|       54510| 3.428977494999999E7|      4.12|
|4a3ca9315b744ce9f...|      330659| 3.375938104000014E7|      3.77|
|7c67e1448b00f6e96...|      233303| 3.228194686999958E7|      3.42|
|fa1c13f2614d7b5c4...|       87679| 3.013763311000003E7|      4.38|
|da8622b14eb17ae28...|      264431|2.9857519929999955E7|      3.98|
|7e93a43ef30c4f03f...|       50223| 2.631430130999988E7|      4.15|
|1025f0e2d44d7041d...|      229582|2.2936882719999965E7|      3.89|
|46dc3b2cc0980fb8e...|       90426| 2.179177328999985E7|      4.18|
|955fee9216a65b617...|      232360| 2.096402587000002E7|      4.04|
|7a67c85e85bb2ce85...|      167228|  2.03124429199999E7|      4.26|
|620c87c171fb2a6dd...|      142229| 2.0119543900

In [23]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [63]:
full_orders_df.select('price').show(1)

+-----+
|price|
+-----+
|29.89|
+-----+
only showing top 1 row



In [40]:
# Joins - Remove Ambiguity 

full_orders_df.select(full_orders_df.customer_id).show(1)

+--------------------+
|         customer_id|
+--------------------+
|0881e1574aa6b621c...|
+--------------------+
only showing top 1 row



amazing now its clear I never knew it was a class object and was always confusing even chatgpt couldn't help me understand

In [ ]:
s1 = 5
s1

In [41]:
full_orders_df.select(col('customer_id')).show(1)

+--------------------+
|         customer_id|
+--------------------+
|0881e1574aa6b621c...|
+--------------------+
only showing top 1 row



In [51]:
full_orders_df[['customer_id']].show(1)

+--------------------+
|         customer_id|
+--------------------+
|0881e1574aa6b621c...|
+--------------------+
only showing top 1 row



In [60]:
full_orders_df.selectExpr('price + 5 as new_price').show(5)

+---------+
|new_price|
+---------+
|    34.89|
|    34.89|
|    34.89|
|    34.89|
|    34.89|
+---------+
only showing top 5 rows



In [ ]:
full_df.selectExpr("price + 5 as price").orderBy("price").show()

In [64]:
full_orders_df.createOrReplaceTempView('full_orders')

Just curious to know, aren’t cloud data warehouses not capable of performing aggregations? Why spark is better ?

# Window Function

In [67]:
from pyspark.sql.window import Window

In [68]:
window_spec = Window.partitionBy('seller_id').orderBy(desc('price'))

In [80]:
# Top selling Product by each seller

top_product_seller = full_orders_df.withColumn('dense_rank',row_number().over(window_spec)).filter(col('dense_rank')<=5)

In [81]:
top_product_seller.select('seller_id','product_id','price','dense_rank').orderBy('seller_id','price').show()

+--------------------+--------------------+-----+----------+
|           seller_id|          product_id|price|dense_rank|
+--------------------+--------------------+-----+----------+
|0015a82c2db000af6...|a2ff5a97bf95719e3...|895.0|         3|
|0015a82c2db000af6...|a2ff5a97bf95719e3...|895.0|         1|
|0015a82c2db000af6...|a2ff5a97bf95719e3...|895.0|         4|
|0015a82c2db000af6...|a2ff5a97bf95719e3...|895.0|         2|
|0015a82c2db000af6...|a2ff5a97bf95719e3...|895.0|         5|
|001cca7ae9ae17fb1...|e251ebd2858be1aa7...|199.0|         1|
|001cca7ae9ae17fb1...|e251ebd2858be1aa7...|199.0|         5|
|001cca7ae9ae17fb1...|e251ebd2858be1aa7...|199.0|         2|
|001cca7ae9ae17fb1...|e251ebd2858be1aa7...|199.0|         4|
|001cca7ae9ae17fb1...|e251ebd2858be1aa7...|199.0|         3|
|001e6ad469a905060...|093cd981b714bcdff...|250.0|         2|
|001e6ad469a905060...|093cd981b714bcdff...|250.0|         3|
|001e6ad469a905060...|093cd981b714bcdff...|250.0|         4|
|001e6ad469a905060...|09

# Advance Aggregation

In [84]:
# Product Popularity Metrics

# total_sales, total_revenue, avg_price,price_volatility, unique_seller

product_metrics_df = full_orders_df.groupBy('product_id')\
.agg(
 
    count('order_id').alias('total_sales'),
    sum('price').alias('total_revenue'),
    round(avg('price'),2).alias('avg_price'),
    round(stddev('price'),2).alias('price_volatility'),
    collect_set('seller_id').alias('unique_sellers')
).orderBy(desc('total_sales'))

In [86]:
product_metrics_df.show(truncate=False)

+--------------------------------+-----------+------------------+---------+----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id                      |total_sales|total_revenue     |avg_price|price_volatility|unique_sellers                                                                                                                                                            |
+--------------------------------+-----------+------------------+---------+----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|aca2eb7d00ea1a7b8ebd4e68314663af|86738      |6164450.500000012 |71.07    |3.17            |[955fee9216a65b617aa5c0531780ce60]                                                                                  

In [ ]:
# Monthly Revenue & Order Count Trend

total_orders
total_revenue
aov
min 
max

# Enrichment

In [87]:
full_orders_df[['order_status']].distinct().show()

+------------+
|order_status|
+------------+
|     shipped|
|    canceled|
|    invoiced|
|   delivered|
|  processing|
| unavailable|
|    approved|
+------------+



In [96]:
#Order Status Flags

full_orders_df = full_orders_df.withColumn('is_delivered',when(col('order_status')=='delivered',lit(1)).otherwise(lit(0)))

In [97]:
full_orders_df[['order_status','is_delivered']].distinct().show()

+------------+------------+
|order_status|is_delivered|
+------------+------------+
|     shipped|           0|
|    invoiced|           0|
|   delivered|           1|
|    canceled|           0|
|  processing|           0|
|    approved|           0|
| unavailable|           0|
+------------+------------+



In [98]:
full_orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [ ]:
# HW

total cost and fright calculation

In [100]:
full_orders_df.select('price','freight_value').distinct().show()

+-----+-------------+
|price|freight_value|
+-----+-------------+
| 35.9|         7.78|
|180.0|        15.01|
|58.55|        12.54|
|139.0|        23.55|
|55.13|        15.14|
| 30.5|         7.78|
|399.9|       173.43|
|224.9|        22.06|
| 24.0|         9.34|
|199.0|        20.63|
| 92.0|        17.92|
| 65.0|        15.56|
|68.99|         8.29|
|122.9|         8.21|
| 75.0|        44.45|
|  6.0|         8.72|
| 45.9|        16.11|
| 49.0|         14.1|
|122.3|        18.74|
| 49.9|         8.72|
+-----+-------------+
only showing top 20 rows



In [102]:
full_orders_df.show(1)

+--------------------+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+-------------------+-----+-------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+----------------------+-----------+------------+--------------------+------------------------+-------------+--------------+---------------------------+------------------+------------------+----------------+-----------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+------------------+------------+--------------------+-------------+------------+
|            order_id|         customer_id|           seller_id|          product_id|order_status|order_purchase_timestamp|  ord

In [103]:
customers_spending_df.show(1)

NameError: name 'customers_spending_df' is not defined

In [108]:
customers_spending_df = customers_spending_df.withColumn(
    'customer_segment',
    when(col('AOV')>= 1500,'High_value')
    .when((col('AOV')>=700) & (col('AOV')<1500),'Medium_value')
    .otherwise('Low_value'))

In [109]:
customers_spending_df.show()

+--------------------+------------+------------------+-------+----------------+
|         customer_id|total_orders|     total_revenue|    AOV|customer_segment|
+--------------------+------------+------------------+-------+----------------+
|d3e82ccec3cb5f956...|        6876|         6662844.0|  969.0|    Medium_value|
|df55c14d1476a9a34...|         743|         3565657.0| 4799.0|      High_value|
|fe5113a38e3575c04...|        2292|         3293604.0| 1437.0|    Medium_value|
|ec5b2ba62e5743423...|        1428|         2556120.0| 1790.0|      High_value|
|63b964e79dee32a35...|        6072|         2501664.0|  412.0|       Low_value|
|46bb3c0b1a65c8399...|         748|         2336752.0| 3124.0|      High_value|
|05455dfa7cd02f13d...|        2184| 2160194.400000087|  989.1|    Medium_value|
|3690e975641f01bd0...|         802|         2124498.0| 2649.0|      High_value|
|349509b216bd5ec11...|         743|         1923627.0| 2589.0|      High_value|
|695476b5848d64ba0...|         687|18205

In [111]:
full_orders_df = full_orders_df.join(customers_spending_df.select('customer_id','customer_segment'),'customer_id','left')

In [112]:
full_orders_df['customer_id','customer_segment'].show()

+--------------------+----------------+
|         customer_id|customer_segment|
+--------------------+----------------+
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
|0881e1574aa6b621c...|       Low_value|
+--------------------+----------------+
only showing top 20 rows



In [113]:
full_orders_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (null

In [ ]:
hadoop fs - 
create a external table
save it to some DB or lake